In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Mai 20

@author: yaning
"""

import importlib
import numpy as np
import matplotlib.pyplot as plt
import itertools
import pickle
import math
import random

# my own class files
import weight_normalise.Receptors as Receptors
import weight_normalise.Network as Network
import weight_normalise.create_update_synapse as cus


importlib.reload(Receptors)
importlib.reload(Network)
importlib.reload(cus)

path = "/home/yaning/Documents/Spiking_NN/"
np.set_printoptions(threshold=np.inf)

In [ ]:
with open(path + "datasets/fire_data_10p_8f_non_zero.pkl", "rb") as f:
    fire_data = pickle.load(f)

In [3]:
fire_data.shape

(100, 10, 10, 8, 6000)

In [4]:
pointCount = 6000
# deltaTms = 0.05
# times = np.arange(pointCount) * deltaTms
# initial_Vm = 1.3458754117369027

Input neurons

In [14]:
pixel = 10
feature = 8
input_array = np.empty([pixel,pixel,feature], dtype=object)

for i in range(pixel):
    for j in range(pixel):
        for k in range(feature):
            input_array[i,j,k] = Network.Neuron((i,j,k))

# not sure if need to flatten it 
input_array_flat = input_array.flatten()

Excitatory Neurons

In [15]:
E_height = 20
E_width = 20

E_array = np.empty([E_height, E_width], dtype=object)

for i in range(E_height):
    for j in range(E_width):
        E_array[i,j] = Network.Neuron((i,j))

E_array_flat = E_array.flatten()

Inhibitory Neurons

In [19]:
I_amount = 10

I_array = np.empty(I_amount, dtype=object)
for i in range(I_amount):
    I_array[i] = Network.Neuron((i))

Ouput Neurons

In [22]:
output_size = 5

output_array = np.empty([output_size, output_size], dtype=object)

for i in range(output_size):
    for j in range(output_size):
        output_array[i,j] = Network.Neuron((i,j))
        
output_array_flat = output_array.flatten()

Connection them!

In [ ]:
# Input to E (all-to-all connection) This might not be all to all connection 
# because from V1 to V2 the topological features hold
for i in input_array_flat:
    for j in E_array_flat:
        cus.create_synapse(i, j, "AMPA")

# E to I p=0.1
for i in E_array_flat:
    for j in I_array:
        if random.random() < 0.1:
            cus.create_synapse(i, j, "AMPA+NMDA")

# I to I p=0.1 (should i prevent self connection? they exist but rare)
for i in I_array:
    for j in I_array:
        if random.random() < 0.1:
            cus.create_synapse(i, j, "GABA")

# I to E p=0.024
for i in I_array:
    for j in E_array_flat:
        if random.random() < 0.024:
            cus.create_synapse(i, j, "GABA")

# # E to output (subsets)
stride = 2
kernel_size = 2

for i in output_array_flat:
    idx = i.id
    x_num = idx[0]
    y_num = idx[1]
    connect_arrary = []
    x_select = E_array[stride*x_num:stride*x_num+kernel_size]
    for j in range(len(x_select)):
        connect_arrary.append(x_select[j][stride*y_num:stride*y_num+kernel_size])
    
    connect_arrary = list(itertools.chain.from_iterable(connect_arrary))
    # connect all the selected E neurons
    for c in connect_arrary:
        cus.create_synapse(c, i, "AMPA")

Input neuron firing sequences

Run

2719